# Hyperparameter optimization for Deep RL using Optuma
Diego Minguzzi ([profile](https://www.linkedin.com/in/diego-minguzzi-2775b78/))

Online course [Deep Reinforcement Learning](https://huggingface.co/learn/deep-rl-course), by Hugging Face: exercise of the [Bonus Unit 2](https://huggingface.co/learn/deep-rl-course/unitbonus2/introduction).
<br>
Adapted from Antonin’s Raffin ICRA 2022 presentations:

*  [The tutorial on YouTube](https://youtu.be/ihP7E76KGOI)
*  [The related Colab Notebook](https://colab.research.google.com/github/araffin/tools-for-robotic-rl-icra2022/blob/main/notebooks/optuna_lab.ipynb)


See [Optuna](https://optuna.org/) and the [documentation](https://optuna.readthedocs.io/en/stable/index.html)

The Gym Lunar lander environment is used.<br>

## Lunar Lander environment

## Imports and package installation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt install swig cmake
!pip install gymnasium[box2d]
!pip install huggingface_sb3
!pip install stable-baselines3
!pip install sb3-contrib
!pip install optuna
!pip install optuna-dashboard
!pip install jupyterlab jupyterlab-optuna

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
import gymnasium as gym
import collections
import json
import math
import numpy as np
import os.path
import pickle
import torch
from typing import Any, Dict
import torch.nn as nn

In [ ]:
from stable_baselines3 import PPO, A2C, SAC, TD3, DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback, CallbackList

In [ ]:
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_intermediate_values

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Set all parameters

In [ ]:
import logging as log
LOG_FORMAT_STRING = "%(asctime)s [%(levelname)-7s] %(message)s"
LOG_LEVEL= log.DEBUG
LOG_ROOT_LOGGER = ''

N_TRIALS =          15 #50 # Maximum number of trials
N_JOBS =            1 # Number of jobs to run in parallel
N_STARTUP_TRIALS=   3 # Stop random sampling after N_STARTUP_TRIALS
N_EVALUATIONS=      6
N_TIMESTEPS =       int(300000)  # Training budget
EVAL_FREQ =         int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_ENVS =       4
N_EVAL_EPISODES =   8
TIMEOUT =           int(60 * 60 * 5) # Expressed into seconds.
NUM_STEPS_LR_UPDATE=5000 # After this many steps the learning rate is updated
LR_UPDATE_FACTOR =  0.95 # Learning rate update factor
POLICY=             'MlpPolicy'
IS_DETERMINISTIC=   True
IS_VERBOSE=         0
N_WARMUP_STEPS=     N_EVALUATIONS // 3  # Do not prune before the warmup steps.
SAVED_PARAMS_FILE=  'lunar_lander_params.json'
SAVED_SNAPSHOT_FILE='opt_lunar_lander_study.pkl'
SEED =              95
ENV_ID =            'LunarLander-v2'
BEST_MODEL_FILE=    f'{ENV_ID}.model'
RESULT_OBJECTIVE=   375.0

HyperParams= collections.namedtuple('HyperParams', ['gamma',
                                                    'learning_rate',
                                                    'max_grad_norm',
                                                    'n_epochs',
                                                    'n_steps',
                                                    'ent_coef',
                                                    'batch_size'])
# Guess Hyperparameters, used in the trials after GUESS_HYPERPARAMS_AFTER_N_TRIAL
guess_hyperparams=[ HyperParams(gamma = 1.0 - 0.0016617315216418454,
                                learning_rate = math.pow(10., -2.682161433845602),
                                max_grad_norm = 0.3733377073391161,
                                n_epochs= 5,
                                n_steps = 2 ** 12,
                                ent_coef= 0.01,
                                batch_size=64),
                    HyperParams(gamma = 1.0 - 0.00010534109595607163,
                                learning_rate = math.pow(10., -2.7388359550396095),
                                max_grad_norm = 0.30386280137092286,
                                n_epochs= 5,
                                n_steps = 2 ** 12,
                                ent_coef= 0.01,
                                batch_size=64)
]

GUESS_HYPERPARAMS_AFTER_N_TRIAL=10

In [ ]:
log.basicConfig(level=LOG_LEVEL, format=LOG_FORMAT_STRING)
log.getLogger().setLevel(LOG_LEVEL)
roothandler= log.getLogger().handlers[0]
roothandler.setFormatter( log.Formatter(LOG_FORMAT_STRING) )

In [ ]:
DEFAULT_HYPERPARAMS = {
    "policy": POLICY,
    "env": ENV_ID,
}

In [ ]:
np.random.seed(SEED)

In [ ]:
class DecreaseLearningRateCallback(BaseCallback):
    """ Custom callback to decrease the learning rate periodically during training. """

    def __init__(self, decay_interval, decay_factor):
        super(DecreaseLearningRateCallback, self).__init__()
        self.decay_interval = decay_interval  # Interval at which to decrease the learning rate
        self.decay_factor = decay_factor      # Factor by which to decrease the learning rate

    def _on_step(self) -> bool:
        if 0==(self.num_timesteps % self.decay_interval):
            # Retrieve the current learning rate
            current_lr = self.model.learning_rate

            # Decrease the learning rate by the decay factor
            new_lr = current_lr * self.decay_factor

            # Set the new learning rate
            self.model.learning_rate = new_lr
            if 0==(self.num_timesteps % (10*self.decay_interval)):
              log.debug(f'Num steps:{self.num_timesteps} current_lr:{current_lr} new_lr:{new_lr}')

        return True

### Define the sampling function



In [ ]:
def sample_ppo_params(trial: optuna.Trial) -> Dict[str, Any]:
    """ Samples the model hyperparameters.
    :param trial: Optuna trial object
    :return: The sampled hyperparameters for the given trial.
    """
    indx_trial= trial.number
    if ( (indx_trial>=GUESS_HYPERPARAMS_AFTER_N_TRIAL)
      and (indx_trial - GUESS_HYPERPARAMS_AFTER_N_TRIAL) < len(guess_hyperparams) ):
      indx_guess= indx_trial - GUESS_HYPERPARAMS_AFTER_N_TRIAL
      log.info(f'indx_trial:{indx_guess} indx_guess:{indx_guess}: using guessed params.')
      gamma = guess_hyperparams[indx_guess].gamma
      learning_rate = guess_hyperparams[indx_guess].learning_rate
      max_grad_norm = guess_hyperparams[indx_guess].max_grad_norm
      n_epochs = guess_hyperparams[indx_guess].n_epochs
      n_steps = guess_hyperparams[indx_guess].n_steps
      ent_coef= guess_hyperparams[indx_guess].ent_coef
      batch_size= guess_hyperparams[indx_guess].batch_size
    else:
      gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.2, log=True)
      learning_rate = math.pow(10., trial.suggest_float("learning_rate_exp", -4.0, -1.0))
      max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 10.0, log=True)
      n_epochs= trial.suggest_int("n_epochs", 2, 16)
      n_steps = 2 ** trial.suggest_int("exponent_n_steps", 6, 13)

    ent_coef= guess_hyperparams[-1].ent_coef
    batch_size= guess_hyperparams[-1].batch_size

    result= {
        "ent_coef": ent_coef,
        "n_steps": n_steps,
        "gamma": gamma,
        "learning_rate": learning_rate,
        "max_grad_norm": max_grad_norm,
        "n_epochs":n_epochs,
        "batch_size":batch_size
    }
    log.info(f'sample_ppo_params() {trial.number}\nParams:{result}')
    return result

### Integrate the performance evaluation in stable baselines.

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback

class TrialEvalCallback(EvalCallback):
    """
    Callback used for evaluating and reporting a trial.

    :param eval_env: Evaluation environement
    :param trial: Optuna trial object
    :param n_eval_episodes: Number of evaluation episodes
    :param eval_freq:   Evaluate the agent every ``eval_freq`` call of the callback.
    :param deterministic: Whether the evaluation should
        use a stochastic or deterministic policy.
    :param verbose:
    """
    def __init__(self,
                  eval_env: gym.Env,
                  trial: optuna.Trial,
                  n_eval_episodes: int = 5,
                  eval_freq: int = 10000,
                  deterministic: bool = IS_DETERMINISTIC,
                  verbose: int = IS_VERBOSE):

        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            # Evaluate policy (done in the parent class)
            super()._on_step()
            self.eval_idx += 1
            # Send report to Optuna
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True

### Define the objective function: it evaluates a set of hyperparameters.

In [ ]:
def objective(trial: optuna.Trial) -> float:
    """
    Objective function using by Optuna to evaluate
    one configuration (i.e., one set of hyperparameters).

    Given a trial object, it will sample hyperparameters,
    evaluate it and report the result (mean episodic reward after training)

    :param trial: Optuna trial object
    :return: Mean episodic reward after training
    """
    print(f'objective started trial:{trial.number}.')
    kwargs = DEFAULT_HYPERPARAMS.copy()

    # 1. Sample hyperparameters and update the keyword arguments
    sampled_params= sample_ppo_params( trial)
    kwargs.update( sampled_params)

    # Create the RL model
    model = PPO(**kwargs)

    # 2. Create envs used for evaluation using `make_vec_env`, `ENV_ID` and `N_EVAL_ENVS`
    eval_env= make_vec_env(ENV_ID, N_EVAL_ENVS)

    # 3. Create the `TrialEvalCallback` callback defined above that will periodically evaluate
    eval_callback = TrialEvalCallback( eval_env, trial, N_EVAL_EPISODES, EVAL_FREQ, deterministic=IS_DETERMINISTIC)
    lr_callback = DecreaseLearningRateCallback(decay_interval=NUM_STEPS_LR_UPDATE, decay_factor=LR_UPDATE_FACTOR)
    list_callbacks = CallbackList([eval_callback, lr_callback])

    nan_encountered = False
    try:
        # Train the model
        model.learn(N_TIMESTEPS, callback=list_callbacks)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN
        log.error('Exception in objective:{e}')
        nan_encountered = True
    finally:
        # Free memory
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    mean_reward, std_reward = evaluate_policy(model, eval_env, N_EVAL_EPISODES, deterministic=IS_DETERMINISTIC)
    result = mean_reward-std_reward
    log.info(f'objective: trial:{trial.number}\nMean reward:{mean_reward} std_reward:{std_reward} result:{result}')

    if (0==trial.number) or ( (trial.number>0) and (result > trial.study.best_value) ):
      model.save(BEST_MODEL_FILE)
      log.info(f'Saved best model to file:{BEST_MODEL_FILE}')

    eval_callback.mean_minus_std= result
    return result

### Run the Optimization loop

The optimization loop runs for a given number of trials, or until the TIMEOUT expires.<br>
Then, a snapshot file is saved.  If the snapshot exists, it is loaded before starting the optimization.

In [ ]:
import torch as th

# Set pytorch num threads to 1 for faster training
th.set_num_threads(1)

# Select the sampler, can be random, TPESampler, CMAES, ...
sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)

pruner = MedianPruner( n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_WARMUP_STEPS )

# Create the study and start the hyperparameter optimization
if not os.path.exists(SAVED_SNAPSHOT_FILE):
  log.info('Started the study from scratch.')
  study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")
else:
  log.info(f'Resuming the study from the file: {SAVED_SNAPSHOT_FILE}')
  snapshot_file = open(SAVED_SNAPSHOT_FILE, 'rb')
  with open(SAVED_SNAPSHOT_FILE, 'rb') as snapshot_file:
    study = pickle.load(snapshot_file)

try:
  study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT)
except Exception as e:
  log.error(f'Exception during study: {e}')
except KeyboardInterrupt:
    pass

snapshot_file = open(SAVED_SNAPSHOT_FILE, 'wb')
pickle.dump(study, snapshot_file)
snapshot_file.close()

log.info(f'Saved the study to the file: {os.path.abspath(SAVED_SNAPSHOT_FILE)}')

2023-12-31 13:48:24,630 [INFO   ] Started the study from scratch.
[I 2023-12-31 13:48:24,632] A new study created in memory with name: no-name-74537ef2-8629-4266-94d6-a838dc9030b1
2023-12-31 13:48:24,637 [INFO   ] sample_ppo_params() 0
Params:{'ent_coef': 0.01, 'n_steps': 8192, 'gamma': 0.999896735134487, 'learning_rate': 0.00017432784789502684, 'max_grad_norm': 1.309833970119273, 'n_epochs': 6, 'batch_size': 64}


objective started trial:0.


2023-12-31 13:50:26,827 [DEBUG  ] Num steps:50000 current_lr:0.00010987002323440212 new_lr:0.000104376522072682
2023-12-31 13:52:39,101 [DEBUG  ] Num steps:100000 current_lr:6.578324142541546e-05 new_lr:6.249407935414468e-05
2023-12-31 13:55:22,901 [DEBUG  ] Num steps:150000 current_lr:3.9386856624232566e-05 new_lr:3.741751379302094e-05
2023-12-31 13:58:07,630 [DEBUG  ] Num steps:200000 current_lr:2.358236598141386e-05 new_lr:2.2403247682343167e-05
2023-12-31 14:00:56,411 [DEBUG  ] Num steps:250000 current_lr:1.4119633627711e-05 new_lr:1.341365194632545e-05
2023-12-31 14:04:00,847 [DEBUG  ] Num steps:300000 current_lr:8.45394622142297e-06 new_lr:8.031248910351821e-06
2023-12-31 14:04:28,063 [INFO   ] objective: trial:0
Mean reward:-19.94559 std_reward:70.77520029192992 result:-90.72079029192992
2023-12-31 14:04:28,075 [INFO   ] Saved best model to file:LunarLander-v2.model
[I 2023-12-31 14:04:28,077] Trial 0 finished with value: -90.72079029192992 and parameters: {'gamma': 0.0001032648

objective started trial:1.


2023-12-31 14:07:17,882 [DEBUG  ] Num steps:50000 current_lr:0.0004823142468631785 new_lr:0.00045819853452001955
2023-12-31 14:10:03,417 [DEBUG  ] Num steps:100000 current_lr:0.00028877935591792325 new_lr:0.00027434038812202706
2023-12-31 14:12:47,225 [DEBUG  ] Num steps:150000 current_lr:0.00017290286767752773 new_lr:0.00016425772429365135
2023-12-31 14:15:33,253 [DEBUG  ] Num steps:200000 current_lr:0.00010352333377878134 new_lr:9.834716708984227e-05
2023-12-31 14:18:13,930 [DEBUG  ] Num steps:250000 current_lr:6.198324400646059e-05 new_lr:5.888408180613756e-05
2023-12-31 14:20:53,250 [DEBUG  ] Num steps:300000 current_lr:3.7111657800493794e-05 new_lr:3.52560749104691e-05
2023-12-31 14:20:54,743 [INFO   ] objective: trial:1
Mean reward:192.18145425 std_reward:113.87917190943759 result:78.30228234056241
2023-12-31 14:20:54,756 [INFO   ] Saved best model to file:LunarLander-v2.model
[I 2023-12-31 14:20:54,758] Trial 1 finished with value: 78.30228234056241 and parameters: {'gamma': 0.0

...

2023-12-31 16:17:48,918 [DEBUG  ] Num steps:50000 current_lr:0.002135448000046126 new_lr:0.00202867560004382
2023-12-31 16:20:45,132 [DEBUG  ] Num steps:100000 current_lr:0.0012785715994503345 new_lr:0.0012146430194778176
[I 2023-12-31 16:20:45,134] Trial 12 pruned. 
2023-12-31 16:20:45,162 [INFO   ] sample_ppo_params() 13
Params:{'ent_coef': 0.01, 'n_steps': 64, 'gamma': 0.9991888679252675, 'learning_rate': 0.00037150030997262063, 'max_grad_norm': 0.8055580951601621, 'n_epochs': 14, 'batch_size': 64}


objective started trial:13.


2023-12-31 16:23:40,290 [DEBUG  ] Num steps:50000 current_lr:0.00023413785107275346 new_lr:0.00022243095851911577
2023-12-31 16:26:36,507 [DEBUG  ] Num steps:100000 current_lr:0.00014018698031115172 new_lr:0.00013317763129559412
[I 2023-12-31 16:26:36,511] Trial 13 pruned. 
2023-12-31 16:26:36,550 [INFO   ] sample_ppo_params() 14
Params:{'ent_coef': 0.01, 'n_steps': 2048, 'gamma': 0.9908686275040927, 'learning_rate': 0.018124075726541428, 'max_grad_norm': 2.103731430237438, 'n_epochs': 10, 'batch_size': 64}


objective started trial:14.


2023-12-31 16:28:51,061 [DEBUG  ] Num steps:50000 current_lr:0.011422688028456852 new_lr:0.010851553627034008
2023-12-31 16:30:56,265 [DEBUG  ] Num steps:100000 current_lr:0.0068391852680331254 new_lr:0.006497226004631469
[I 2023-12-31 16:30:56,266] Trial 14 pruned. 
2023-12-31 16:30:56,272 [INFO   ] Saved the study to the file: /content/opt_lunar_lander_study.pkl


In [ ]:
# Write report
study.trials_dataframe().to_csv("study_results_ppo_lunar_lander.csv")

In [ ]:
fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)
fig3 = plot_intermediate_values(study)

fig1.show()
fig2.show()
fig3.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
with open(SAVED_PARAMS_FILE, 'w') as fp:
    json.dump(study.best_params, fp)

In [ ]:
study.best_params

{}

In [ ]:
if len(study.trials)>0:
    trial = study.best_trial
    print(f'Best trial: {trial.number} Value: {trial.value}')

    print("  Params: ")
    for key, value in study.best_params.items():
        print(f"    {key}: {value}")

Best trial: 11 Value: 227.4163204622611
  Params: 
